In [1]:
import numpy as np

In [12]:
X = np.array([[1,2],[3,4], [5,6]])
y = np.array([[10,20,30]]).reshape(-1, 1)

In [15]:
X.T @ y

array([[220],
       [280]])

In [5]:
# import superimport

import numpy as np
import scipy
from scipy import linalg
from itertools import combinations

In [17]:
def naive_solve(A, b):
    return np.linalg.inv(A.T @ A) @ A.T @ b

def qr_solve(A, b):
    m, n = np.shape(A)
    if m > n:
        return qr_solve_over(A, b)
    else:
        return qr_solve_under(A, b)
    
def qr_solve_over(A, b):
    Q, R = np.linalg.qr(A) 
    Qb = np.dot(Q.T,b) 
    return scipy.linalg.solve_triangular(R, Qb)

def qr_solve_under(A, b):
    Q, R = np.linalg.qr(A.T) 
    c = scipy.linalg.solve_triangular(R, b)
    m, n = A.shape
    x = np.zeros(n)
    xx = np.dot(Q, c)
    xx = xx[:,0]
    K = xx.shape[0]
    x[:K] = xx # other components are zero
    return x



def run_demo(m, n):
    print('Solving linear system with {} constraints and {} unknowns'.format(m, n))
    np.random.seed(0)
    A = np.random.rand(m,n)
    b = np.random.rand(m,1) 

    california_data = np.genfromtxt('./california.csv', delimiter=',')
    california_data = california_data[:, [0, -1]]
    california_data
    
    methods = list()
    solns = list()
    
    methods.append('naive')
    solns.append(naive_solve(A, b))
    
    methods.append('pinv')
    solns.append(np.dot(np.linalg.pinv(A), b))
    
    methods.append('lstsq')
    solns.append(np.linalg.lstsq(A, b, rcond=None)[0])
    
    methods.append('qr')
    solns.append(qr_solve(A, b))
    
    
    for (method, soln) in zip(methods, solns):
        residual = b -  np.dot(A, soln)
        print('method {}, norm {:0.5f}, residual {:0.5f}'.format(method, np.linalg.norm(soln), np.linalg.norm(residual)))
        # print(soln.T)
    
    # https://stackoverflow.com/questions/33559946/numpy-vs-mldivide-matlab-operator
    if m < n: # underdetermined
        rank = np.linalg.matrix_rank(A)
        assert m==rank
        for nz in combinations(range(n), rank):    # the variables not set to zero
            soln = np.zeros((n,1))
            soln[nz, :] = np.asarray(np.linalg.solve(A[:, nz], b))
            residual = b -  np.dot(A, soln)
            print('sparse qr, norm {:0.5f}, residual {:0.5f}'.format(np.linalg.norm(soln), np.linalg.norm(residual)))
            print(soln.T) 
            print('\n\n')
    
m = 10000; n = 200 # Overdetermined
run_demo(m, n)

# m = 3; n = 5 # Underdetermined
# run_demo(m, n)

Solving linear system with 10000 constraints and 200 unknowns
method naive, norm 0.16326, residual 28.56347
method pinv, norm 0.16326, residual 28.56347
method lstsq, norm 0.16326, residual 28.56347
method qr, norm 0.16326, residual 28.56347
